In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import bisect
import time, requests, os, xlrd, sys
from datetime import timedelta,date
pd.set_option('precision',3)

hd_date=pd.datetime.today()#历史表的统计结束日期，软件初始默认为当天。pd.datetime.today()'2010-10-4'
#指定日期的指数PE（等权重）

def calc_state(data):#对当前百分位状态的评估
    if data < 10.0:
        return u'极度低估'
    elif 10 <= data  and data < 20:
        return u'低估'
    elif 20 <= data  and data < 40:
        return u'正常偏低'
    elif 40 <= data  and data < 60:
        return u'正常'
    elif 60 <= data  and data < 80:
        return u'正常偏高'
    elif 80 <= data  and data < 90:
        return u'高估'
    elif 90 <= data:
        return u'极度高估'

def get_index_pe_pb_date(code,date):
    '''指定日期的指数PE_PB（等权重）'''
    stocks = get_index_stocks(code, date)#返回指数对应的股票代码
    q = query(valuation).filter(valuation.code.in_(stocks))
    df = get_fundamentals(q, date)#获取指数下对应股票的具体信息列表
    if len(df)>0:
        pe = len(df)/sum([1/p if p>0 else 0 for p in df.pe_ratio])#求等权重pe，len（df）返回股票数
        pb = len(df)/sum([1/p if p>0 else 0 for p in df.pb_ratio])
        return (round(pe,2), round(pb,2))#返回求整数后的pe和pb值
    else:
        return float('NaN')
    
 
#指数历史PEPB
def get_index_pe_pb(code, start_date=None, end_date=None):
    '''指数历史PE_PB'''
    if start_date is None:
        start_date = get_security_info(code).start_date
        if start_date < date(2005,01,01): #只计算2005年以来的数据
            start_date = date(2005,01,01)
    if end_date is None:
        end_date =pd.datetime.today() - timedelta(1)#结束日期为程序运行日期的前一天
    x = get_price(code, start_date=start_date, end_date=end_date, frequency='daily', fields='close')
    date_list = x.index.tolist()
#     print date_list
    pe_list = []
    pb_list = []
    for d in date_list: #交易日
        pe_pb = get_index_pe_pb_date(code,d)
        pe_list.append(pe_pb[0])
        pb_list.append(pe_pb[1])
    df = pd.DataFrame({'PE': pd.Series(pe_list, index=date_list),
                        'PB': pd.Series(pb_list, index=date_list)})
    return df #返回指数“日期，pe，pb”的列表


def get_hs_data(index_choose,data_root='./'):
    '''增量更新沪深指数估值数据'''
    for code in index_choose:
#         print u'正在计算:', code
        data_path = '%s%s_pe_pb.csv'%(data_root,convert_code(code))
        if os.path.exists(data_path):#增量更新
            df_pe_pb = pd.DataFrame.from_csv(data_path)
            start_date = df_pe_pb.iloc[-1].name + timedelta(1)
            df_pe_pb = pd.concat([df_pe_pb, get_index_pe_pb(code, start_date)]) 
        else:#初次计算
            print 'init'
            df_pe_pb = get_index_pe_pb(code)
        df_pe_pb.to_csv(data_path)
        
def convert_code(code):#代码显示模式变更
    if code.endswith('XSHG'):
        return 'sh' + code[0:6]
    elif code.endswith('XSHE'):
        return 'sz' + code[0:6]   



# index_choose =['000016.XSHG',                    
#                          '000300.XSHG',
#                          '399106.XSHE',               
#                          '399316.XSHE',
#                          '000036.XSHG',
#                          '000037.XSHG',
#                          '000038.XSHG',
#                          '000039.XSHG',
#                          '000158.XSHG',
#                          '000827.XSHG',
#                          '000978.XSHG',
#                          '000933.XSHG',
#                          '000905.XSHG'
#                         ]
index_choose =[    
     '000016.XSHG','000300.XSHG','000905.XSHG','000852.XSHG','399006.XSHE',#宽指数
     '000001.XSHG','399001.XSHE','000902.XSHG','000985.XSHG',#大盘指数
     '000015.XSHG','000922.XSHG','000827.XSHG','000978.XSHG',#策略指数
    '000990.XSHG','000991.XSHG',#全指医药消费
     '399967.XSHE','399975.XSHE',#中证军工、证券
    '399106.XSHE','399316.XSHE','000036.XSHG','000037.XSHG','000038.XSHG','000039.XSHG','000158.XSHG','000933.XSHG'
]

all_index = get_all_securities(['index'])
get_hs_data(index_choose) #更新沪深数据
df_pe_pb = pd.DataFrame()
frames1=pd.DataFrame()
frames2=pd.DataFrame()
today=hd_date#pd.datetime.today() #统计的截止日，虚拟当天。用于模拟测试策略。
for code in index_choose:
    data_path = '%s%s_pe_pb.csv'%('./',convert_code(code))
    index_name = all_index.ix[code].display_name
    df_pe_pb = pd.DataFrame.from_csv(data_path)    
    df_pe_pb = df_pe_pb[df_pe_pb.iloc[-1].name.date() - timedelta(365*13):] #最长十年的数据
    if len(df_pe_pb)<250*3: #每年250个交易日,小于3年不具有参考价值
    #                 print code, 'samples:', len(df_pe_pb), index_name
        continue
#     print u'正在处理: ',index_name   
#     df_pe_pb=get_index_pe_pb(code)    
    results1=[]
    pe = get_index_pe_pb_date(code,today)[0]
    q_pes = [df_pe_pb['PE'].quantile(i/10.0)  for i in range(11)]    
    idx = bisect.bisect(q_pes,pe)
    quantile = idx-(q_pes[idx]-pe)/(q_pes[idx]-q_pes[idx-1])   
    
    results1.append([code[0:6],index_name,'%.2f'% pe,'%.2f'% (quantile*10)]+['%.2f'%q  for q in q_pes]+[
                    calc_state(quantile*10),'%.2f'%median(df_pe_pb.PE), '%.2f'%std(df_pe_pb.PE),
                    df_pe_pb.iloc[0].name.date()]+[df_pe_pb['PE'].count()])
    
    results2=[]
    pb = get_index_pe_pb_date(code,today)[1]
    q_pbs = [df_pe_pb['PB'].quantile(i/10.0)  for i in range(11)] 
    idx = bisect.bisect(q_pbs,pb)
    quantile = idx-(q_pbs[idx]-pb)/(q_pbs[idx]-q_pbs[idx-1])   
    #index_name = all_index.ix[code].display_name
    results2.append([code[0:6],index_name,'%.2f'% pb,'%.2f'% (quantile*10)]+['%.2f'%q  for q in q_pbs]+[calc_state(quantile*10),
                       '%.2f'%median(df_pe_pb.PB),'%.2f'%std(df_pe_pb.PB),
                       df_pe_pb.iloc[0].name.date()]+[df_pe_pb['PB'].count()])

    
    df_pe_pb['10% PE']=q_pes[1]
    df_pe_pb['50% PE']=q_pes[5]
    df_pe_pb['90% PE']=q_pes[9]
    df_pe_pb['10% PB']=q_pbs[1]
    df_pe_pb['50% PB']=q_pbs[5]
    df_pe_pb['90% PB']=q_pbs[9]
    
#选择图片输出的指数
#     if code in ['000001.XSHG','000016.XSHG','000300.XSHG','000905.XSHG','399006.XSHE',\
#                 '000827.XSHG','000978.XSHG','399967.XSHE','000933.XSHG']:
#         df_pe_pb.plot(secondary_y=['PB','10% PB','50% PB','90% PB'],\
#                   figsize=(14,8),title=index_name,style=['k-.', 'k', 'g', 'y', 'r', 'g-.', 'y-.', 'r-.']) 
        
    columns=[u'code',u'名称',u'当前估值',u'分位点%',u'最小估值']+['%d%%'% (i*10) for i in range(1,10)]+\
    [u'最大估值',u'计算状态',u'中位数',u'标准差',u'起始日期',u"数据个数"]
    
    df1= pd.DataFrame(data=results1,index=['PE'],columns=columns)#改装PE，PB两个表分开显示，对应results，df，和frames均进行了区分
    frames1 = pd.concat([frames1, df1])  
    df2= pd.DataFrame(data=results2,index=['PB'],columns=columns)
    frames2 = pd.concat([frames2, df2])  


In [2]:
frames1[[u'当前估值',u'分位点%']]=frames1[[u'当前估值',u'分位点%']].astype(float)
frames1=frames1.sort(columns=u'分位点%',ascending=True)
frames1.iloc[:,[0,1,2,3,4,14,15,16,17,18]]

,code,名称,当前估值,分位点%,最小估值,最大估值,计算状态,中位数,标准差,起始日期
PE,000852,中证1000指数,32.53,0.00,32.53,118.16,极度低估,59.44,14.99,2014-10-17
PE,000158,上证环保,23.43,0.00,23.43,63.98,极度低估,34.45,5.76,2012-09-25
PE,000827,中证环保,26.44,0.55,26.14,75.04,极度低估,36.46,7.09,2012-09-25
PE,399316,巨潮小盘,26.88,7.27,18.05,91.93,极度低估,40.73,15.09,2005-05-09
PE,000978,医药100,30.99,8.86,26.40,70.90,极度低估,36.81,6.24,2011-03-18
PE,000037,上证医药,28.12,8.95,21.73,66.67,极度低估,33.78,5.65,2009-01-09
PE,000905,中证500,25.55,9.73,16.74,79.09,极度低估,36.33,13.25,2007-01-15
PE,000933,中证医药,28.53,12.27,22.22,67.69,低估,34.79,6.39,2009-07-03
PE,399967,中证军工,69.38,14.16,56.32,199.46,低估,92.40,29.04,2013-12-26
PE,000922,中证红利,12.88,15.57,9.15,43.59,低估,15.19,6.50,2008-05-26


In [3]:
frames2[[u'当前估值',u'分位点%']]=frames2[[u'当前估值',u'分位点%']].astype(float)
frames2=frames2.sort(columns=u'分位点%',ascending=True)
frames2.iloc[:,[0,1,2,3,4,14,15,16,17,18]]

,code,名称,当前估值,分位点%,最小估值,最大估值,计算状态,中位数,标准差,起始日期
PB,399975,中证全指证券公司指数(四级行业),1.51,0.29,1.50,5.54,极度低估,2.16,0.88,2013-07-15
PB,000852,中证1000指数,2.45,1.52,2.38,7.25,极度低估,3.71,0.77,2014-10-17
PB,000038,上证金融,1.17,7.69,1.07,4.10,极度低估,1.48,0.66,2009-01-09
PB,399967,中证军工,3.13,8.47,2.63,9.83,极度低估,4.59,1.27,2013-12-26
PB,000990,全指消费,2.71,8.61,2.40,6.46,极度低估,3.32,0.60,2011-08-02
PB,000037,上证医药,3.23,8.80,2.79,6.86,极度低估,3.70,0.77,2009-01-09
PB,000827,中证环保,2.32,8.99,1.82,6.53,极度低估,2.99,0.71,2012-09-25
PB,000922,中证红利,1.40,9.09,1.20,3.18,极度低估,1.72,0.46,2008-05-26
PB,000015,红利指数,1.25,9.23,1.13,5.04,极度低估,1.55,0.73,2005-05-09
PB,000016,上证50,1.41,9.33,1.27,6.64,极度低估,1.81,1.06,2005-05-09


In [4]:
import pandas as pd
pd.options.mode.chained_assignment = None

def np_sum(index_symbol,date, pStatDate):
    stocks = get_index_stocks(index_symbol, date)
    q = query(income).filter(income.code.in_(stocks))
    #取一个季度的数据
    df = get_fundamentals(q, statDate=pStatDate)
    if len(df):
        npsum=sum(p for p in df.np_parent_company_owners)
        return npsum
    else:
        return float('NaN')
    
# 修正后让函数返回3个值：年同比增长yr，季度环比增长seah，季度同比增长seat。
def year_return(index_symbol,date):
    date = pd.to_datetime(date)
    curMonth=date.month
    if curMonth in [1,2,3]:
        #Q1
        #取得上年Q4,Q3,Q2,Q1
        curQ4=str(date.year-1)+"q4"
        curQ3=str(date.year-1)+"q3"
        curQ2=str(date.year-1)+"q2"
        curQ1=str(date.year-1)+"q1"
        curQ4Np=np_sum(index_symbol,date,curQ4)
        curQ3Np=np_sum(index_symbol,date,curQ3)
        curQ2Np=np_sum(index_symbol,date,curQ2)
        curQ1Np=np_sum(index_symbol,date,curQ1)

        curNp=curQ4Np+curQ3Np+curQ2Np+curQ1Np

        #取得上上年Q4,Q3,Q2,Q1
        lasQ4=str(date.year-2)+"q4"
        lasQ3=str(date.year-2)+"q3"
        lasQ2=str(date.year-2)+"q2"
        lasQ1=str(date.year-2)+"q1"
        lasQ4Np=np_sum(index_symbol,date,lasQ4)
        lasQ3Np=np_sum(index_symbol,date,lasQ3)
        lasQ2Np=np_sum(index_symbol,date,lasQ2)
        lasQ1Np=np_sum(index_symbol,date,lasQ1)

        lasNp=lasQ4Np+lasQ3Np+lasQ2Np+lasQ1Np
        
        yr=(curNp-lasNp)/lasNp
        sea_h=(curQ4Np-curQ3Np)/curQ3Np
        sea_t=(curQ4Np-lasQ4Np)/lasQ4Np

        return (yr,sea_h,sea_t)
    elif curMonth in [4,5,6]:
        #Q2
        #取得当年Q1,上年Q4,Q3,Q2
        curQ4=str(date.year)+"q1"
        curQ3=str(date.year-1)+"q4"
        curQ2=str(date.year-1)+"q3"
        curQ1=str(date.year-1)+"q2"        
        curQ4Np=np_sum(index_symbol,date,curQ4)
        curQ3Np=np_sum(index_symbol,date,curQ3)
        curQ2Np=np_sum(index_symbol,date,curQ2)
        curQ1Np=np_sum(index_symbol,date,curQ1)

        curNp=curQ4Np+curQ3Np+curQ2Np+curQ1Np

        #取得上年Q1,上上年Q4,Q3,Q2
        lasQ4=str(date.year-1)+"q1"
        lasQ3=str(date.year-2)+"q4"
        lasQ2=str(date.year-2)+"q3"
        lasQ1=str(date.year-2)+"q2"        

        lasQ4Np=np_sum(index_symbol,date,lasQ4)
        lasQ3Np=np_sum(index_symbol,date,lasQ3)
        lasQ2Np=np_sum(index_symbol,date,lasQ2)
        lasQ1Np=np_sum(index_symbol,date,lasQ1)

        lasNp=lasQ4Np+lasQ3Np+lasQ2Np+lasQ1Np

        yr=(curNp-lasNp)/lasNp
        sea_h=(curQ4Np-curQ3Np)/curQ3Np
        sea_t=(curQ4Np-lasQ4Np)/lasQ4Np

        return (yr,sea_h,sea_t)
    elif curMonth in [7,8,9]:
        #Q3
        #取得当年Q2,Q1,上年Q4,Q3
        curQ4=str(date.year)+"q2"
        curQ3=str(date.year)+"q1"
        curQ2=str(date.year-1)+"q4"
        curQ1=str(date.year-1)+"q3" 

        curQ4Np=np_sum(index_symbol,date,curQ4)
        curQ3Np=np_sum(index_symbol,date,curQ3)
        curQ2Np=np_sum(index_symbol,date,curQ2)
        curQ1Np=np_sum(index_symbol,date,curQ1)

        curNp=curQ4Np+curQ3Np+curQ2Np+curQ1Np


        #取得上年Q2,Q1,上上年Q4,Q3
        lasQ4=str(date.year-1)+"q2"
        lasQ3=str(date.year-1)+"q1"
        lasQ2=str(date.year-2)+"q4"
        lasQ1=str(date.year-2)+"q3"  

        lasQ4Np=np_sum(index_symbol,date,lasQ4)
        lasQ3Np=np_sum(index_symbol,date,lasQ3)
        lasQ2Np=np_sum(index_symbol,date,lasQ2)
        lasQ1Np=np_sum(index_symbol,date,lasQ1)

        lasNp=lasQ4Np+lasQ3Np+lasQ2Np+lasQ1Np
        
        yr=(curNp-lasNp)/lasNp
        sea_h=(curQ4Np-curQ3Np)/curQ3Np
        sea_t=(curQ4Np-lasQ4Np)/lasQ4Np

        return (yr,sea_h,sea_t)
    elif curMonth in [10,11,12]:
        #Q4
        #取得当年Q3,Q2,Q1,上年Q4
        curQ4=str(date.year)+"q3"
        curQ3=str(date.year)+"q2"
        curQ2=str(date.year)+"q1"
        curQ1=str(date.year-1)+"q4" 

        curQ4Np=np_sum(index_symbol,date,curQ4)
        curQ3Np=np_sum(index_symbol,date,curQ3)
        curQ2Np=np_sum(index_symbol,date,curQ2)
        curQ1Np=np_sum(index_symbol,date,curQ1)

        curNp=curQ4Np+curQ3Np+curQ2Np+curQ1Np

        #取得上年Q3,Q2,Q1,上上年Q4
        lasQ4=str(date.year-1)+"q3"
        lasQ3=str(date.year-1)+"q2"
        lasQ2=str(date.year-1)+"q1"
        lasQ1=str(date.year-2)+"q4"  

        lasQ4Np=np_sum(index_symbol,date,lasQ4)
        lasQ3Np=np_sum(index_symbol,date,lasQ3)
        lasQ2Np=np_sum(index_symbol,date,lasQ2)
        lasQ1Np=np_sum(index_symbol,date,lasQ1)

        lasNp=lasQ4Np+lasQ3Np+lasQ2Np+lasQ1Np

        yr=(curNp-lasNp)/lasNp
        sea_h=(curQ4Np-curQ3Np)/curQ3Np
        sea_t=(curQ4Np-lasQ4Np)/lasQ4Np

        return (yr,sea_h,sea_t)

def index_pe(index_symbol,date):
    stocks = get_index_stocks(index_symbol, date)
    q = query(valuation).filter(valuation.code.in_(stocks))
    df = get_fundamentals(q, date)
    if len(df)>0:
        pe = len(df)/sum([1/p if p>0 else 0 for p in df.pe_ratio])
        return pe
    else:
        return float('NaN')
    
index_symbols = [ '000016.XSHG','000300.XSHG','000905.XSHG','000852.XSHG','399006.XSHE',#宽指数
     '000001.XSHG','399001.XSHE','000902.XSHG','000985.XSHG',#大盘指数
     '000015.XSHG','000922.XSHG','000827.XSHG','000978.XSHG',#策略指数
    '000990.XSHG','000991.XSHG',#全指医药消费
     '399967.XSHE','399975.XSHE',#中证军工、证券
    '399106.XSHE','399316.XSHE','000036.XSHG','000037.XSHG','000038.XSHG','000039.XSHG','000158.XSHG','000933.XSHG']

allindexs = get_all_securities('index')
df_index = allindexs[allindexs.index.isin(index_symbols)]

date = pd.datetime.today()

for symbol in index_symbols:
    (yr,sea_h,sea_t)= year_return(symbol,date)
    pe = index_pe(symbol,date)
    df_index.loc[symbol,'年增长%'] = '{:.2%}'.format(yr)
    df_index.loc[symbol,'季度环比%']='{:.2%}'.format(sea_h)
    df_index.loc[symbol,'季度同比%']='{:.2%}'.format(sea_t)
    df_index.loc[symbol,'PE'] = '{:.2f}'.format(pe)
    if yr<0:
        df_index.loc[symbol,'PEG_Y'] = float('inf')
    else:
        df_index.loc[symbol,'PEG_Y'] = pe/yr*0.01
    if sea_h<0:
        df_index.loc[symbol,'PEG_H'] = float('inf')
    else:
        df_index.loc[symbol,'PEG_H'] = pe/sea_h*0.01
    if sea_t<0:
        df_index.loc[symbol,'PEG_T'] = float('inf')
    else:
        df_index.loc[symbol,'PEG_T'] = pe/sea_t*0.01
    
    

In [58]:
df1=frames1.sort(columns='code')
df2=frames2.sort(columns='code')
df3=df_index.iloc[:,[0,5,6,7,8,9,10,11,2]]
df1['mix']=(df2[u'分位点%'].values/(df2[u'分位点%'].values.max()-df2[u'分位点%'].values.min())+df1[u'分位点%'].values/\
            (df1[u'分位点%'].values.max()-df1[u'分位点%'].values.min())\
            +df3['PEG_Y'].values/(19.61-0.68))/3
df1['PEG_Y']=df3['PEG_Y'].values
df1=df1.sort(columns='mix')#PE，PB分位点之和平均值排序
df1.iloc[:,[0,1,2,3,4,14,15,16,17,18,20,21]]

,code,名称,当前估值,分位点%,最小估值,最大估值,计算状态,中位数,标准差,起始日期,mix,PEG_Y
PE,000852,中证1000指数,32.53,0.00,32.53,118.16,极度低估,59.44,14.99,2014-10-17,0.03,0.88
PE,000158,上证环保,23.43,0.00,23.43,63.98,极度低估,34.45,5.76,2012-09-25,0.10,0.92
PE,000827,中证环保,26.44,0.55,26.14,75.04,极度低估,36.46,7.09,2012-09-25,0.10,1.22
PE,000037,上证医药,28.12,8.95,21.73,66.67,极度低估,33.78,5.65,2009-01-09,0.14,0.68
PE,000933,中证医药,28.53,12.27,22.22,67.69,低估,34.79,6.39,2009-07-03,0.17,0.90
PE,000922,中证红利,12.88,15.57,9.15,43.59,低估,15.19,6.50,2008-05-26,0.19,1.50
PE,000905,中证500,25.55,9.73,16.74,79.09,极度低估,36.33,13.25,2007-01-15,0.20,0.89
PE,000978,医药100,30.99,8.86,26.40,70.90,极度低估,36.81,6.24,2011-03-18,0.22,0.92
PE,000016,上证50,12.06,24.27,9.57,51.02,正常偏低,14.39,8.62,2005-05-09,0.23,0.90
PE,000036,上证消费,29.89,16.69,22.79,83.33,低估,34.70,14.58,2009-01-09,0.23,0.76


In [57]:
df_index.sort('PEG_Y').iloc[:,[0,5,6,7,8,9,10,11,2]]

,display_name,年增长%,季度环比%,季度同比%,PE,PEG_Y,PEG_H,PEG_T,start_date
000037.XSHG,上证医药,41.50%,27.21%,32.44%,28.12,0.68,1.03,0.87,2009-01-09
000036.XSHG,上证消费,39.39%,46.26%,27.12%,29.89,0.76,0.65,1.10,2009-01-09
399316.XSHE,巨潮小盘,32.40%,-1.01%,21.03%,26.88,0.83,inf,1.28,2005-02-03
000852.XSHG,中证1000指数,36.80%,-4.00%,30.67%,32.53,0.88,inf,1.06,2014-10-17
000038.XSHG,上证金融,10.61%,41.91%,8.39%,9.47,0.89,0.23,1.13,2009-01-09
000905.XSHG,中证500,28.63%,-1.43%,18.56%,25.55,0.89,inf,1.38,2007-01-15
000016.XSHG,上证50,13.45%,35.24%,11.52%,12.06,0.90,0.34,1.05,2004-01-02
000933.XSHG,中证医药,31.74%,23.76%,27.02%,28.53,0.90,1.20,1.06,2009-07-03
000158.XSHG,上证环保,25.51%,-39.27%,4.09%,23.43,0.92,inf,5.73,2012-09-25
000978.XSHG,医药100,33.58%,25.36%,29.67%,30.99,0.92,1.22,1.04,2011-03-18


59.82